In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import arrow
import pandas as pd

Everything is for Spain (ES)

In [7]:
# Installed Capacity
capacity_df = pd.read_csv("raw/Installed Capacity per Production Type_202201010000-202301010000.csv")

capacity_df.loc[:, "Grid Node"] = "ES"
capacity_df.loc[:, "datetime"] = "2022-01-01 00:00:00+00:00"
capacity_df = capacity_df.rename(columns={"Production Type": "Fuel Type", "2022 [MW]": "Value"})

map_fuel_type = {
    "Biomass": "BIOMASS",
    "Fossil Brown coal/Lignite": "BROWN_COAL",
    "Fossil Coal-derived gas": "COAL_DERIVED_GAS",
    "Fossil Gas": "OTHER_GAS",
    "Fossil Hard coal":"HARD_COAL",
    "Fossil Oil shale":"SHALE_OIL",
    "Fossil Oil":"OTHER_OIL",
    "Fossil Peat":"PEAT",
    "Geothermal":"GEOTHERMAL",
    "Hydro Pumped Storage":"HYDRO_PUMPED_STORAGE",
    "Hydro Pumped Storage  - Actual Consumption [MW]":"HYDRO_PUMPED_STORAGE_CONSUMPTION",
    "Hydro Run-of-river and poundage": "HYDRO_RUN_OF_RIVER",
    "Hydro Water Reservoir": "HYDRO_RESERVOIR",
    "Marine": "HYDRO",
    "Nuclear":"NUCLEAR",
    "Other": "OTHER",
    "Other renewable":"OTHER_RENEWABLE",
    "Solar":"SOLAR",
    "Waste":"WASTE",
    "Wind Offshore": "WIND_OFF_SHORE",
    "Wind Onshore":"WIND_ON_SHORE"
}
capacity_df["Fuel Type"] = capacity_df["Fuel Type"].map(map_fuel_type)
capacity_df.loc[:, "unit"] = "MW"
# Remove total, which is the only row without a fuel type
capacity_df = capacity_df[capacity_df["Fuel Type"].notna()]

capacity_df.to_csv("installed_capacity.csv", index=False)

In [4]:
# Generation
generation_df = pd.read_csv("raw/Actual Generation per Production Type_202211010000-202211020000.csv")

generation_df.loc[:, "Grid Node"] = "ES"
generation_df.loc[:, "unit"] = "MWh"

generation_df.loc[:, "datetime"] = generation_df.loc[:, "MTU"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))

generation_df = generation_df.drop(columns=["MTU", "Area"])
generation_df = generation_df.rename(columns={c: c.split("  - Actual Aggregated [MW]")[0] + " (MW)" for c in generation_df.columns if "Actual Aggregated [MW]" in c})
generation_df_out = generation_df.rename(columns={
    "Biomass (MW)": "BIOMASS",
    "Fossil Brown coal/Lignite (MW)": "BROWN_COAL",
    "Fossil Coal-derived gas (MW)": "COAL_DERIVED_GAS",
    "Fossil Gas (MW)": "OTHER_GAS",
    "Fossil Hard coal (MW)":"HARD_COAL",
    "Fossil Oil shale (MW)":"SHALE_OIL",
    "Fossil Oil (MW)":"OTHER_OIL",
    "Fossil Peat (MW)":"PEAT",
    "Geothermal (MW)":"GEOTHERMAL",
    "Hydro Pumped Storage (MW)":"HYDRO_PUMPED_STORAGE",
    "Hydro Pumped Storage  - Actual Consumption [MW]":"HYDRO_PUMPED_STORAGE_CONSUMPTION",
    "Hydro Run-of-river and poundage (MW)": "HYDRO_RUN_OF_RIVER",
    "Hydro Water Reservoir (MW)": "HYDRO_RESERVOIR",
    "Marine (MW)": "HYDRO",
    "Nuclear (MW)":"NUCLEAR",
    "Other (MW)": "OTHER",
    "Other renewable (MW)":"OTHER_RENEWABLE",
    "Solar (MW)":"SOLAR",
    "Waste (MW)":"WASTE",
    "Wind Offshore (MW)": "WIND_OFF_SHORE",
    "Wind Onshore (MW)":"WIND_ON_SHORE"
})

generation_df_out.fillna(0).to_csv("generation.csv", index=False)

In [19]:
# Generation per unit

# ES-ALMARAZ-1
generation_df = pd.read_csv("raw/Actual Generation per Generation Unit_202304150000-202304160000_1.csv")
df1 = pd.DataFrame(columns=["datetime", "Grid Node", "unit", "Fuel Type", "Value"])
df1.datetime = pd.date_range(start="2023-04-15 00:00:00", end="2023-04-16 00:00:00", freq="H", closed="left")
df1["Grid Node"] = "ES-ALMARAZ-1"
df1["unit"] = "MWh"
df1["Fuel Type"] = "NUCLEAR"
df1["value"] = generation_df["Generation [MW]"].values[1:].astype(float)

df1.to_csv("generation_ES-ALMARAZ-1.csv", index=False)

# ES-ALMARAZ-2
generation_df = pd.read_csv("raw/Actual Generation per Generation Unit_202304150000-202304160000_2.csv")
df2 = pd.DataFrame(columns=["datetime", "Grid Node", "unit", "Fuel Type", "Value"])
df2.datetime = pd.date_range(start="2023-04-15 00:00:00", end="2023-04-16 00:00:00", freq="H", closed="left")
df2["Grid Node"] = "ES-ALMARAZ-2"
df2["unit"] = "MWh"
df2["Fuel Type"] = "NUCLEAR"
df2["value"] = generation_df["Generation [MW]"].values[1:].astype(float)

df2.to_csv("generation_ES-ALMARAZ-2.csv", index=False)

# ES-ALMARAZ
# value is the sum of the two units
df = pd.concat([df1, df2])
df = df.groupby("datetime").sum(numeric_only=True).reset_index()
df["Grid Node"] = "ES-ALMARAZ"
df["unit"] = "MWh"
df["Fuel Type"] = "NUCLEAR"

df.to_csv("generation_ES-ALMARAZ.csv", index=False)


/var/folders/qq/pc017vp553z_kn95rv6b8w080000gn/T/ipykernel_55153/3314103856.py:6: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  df1.datetime = pd.date_range(start="2023-04-15 00:00:00", end="2023-04-16 00:00:00", freq="H", closed="left")
/var/folders/qq/pc017vp553z_kn95rv6b8w080000gn/T/ipykernel_55153/3314103856.py:17: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  df2.datetime = pd.date_range(start="2023-04-15 00:00:00", end="2023-04-16 00:00:00", freq="H", closed="left")


In [5]:
# Imports and Exports
imports_exports_df = pd.read_csv("raw/Cross-Border Physical Flow_202211010000-202211020000.csv")


imports_exports_df.loc[:, "datetime"] = imports_exports_df.loc[:, "Time (UTC)"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))

final_columns = ["datetime", "Grid Node From", "Grid Node To", "Value"]
exports_df = imports_exports_df.copy()
imports_df = imports_exports_df.copy()

exports_df.loc[:, "Grid Node From"] = "ES"
exports_df.loc[:, "Grid Node To"] = "FR"
exports_df = exports_df.rename(columns={"BZN|ES > BZN|FR [MW]": "Value"})
exports_df = exports_df[final_columns]
imports_df.loc[:, "Grid Node From"] = "FR"
imports_df.loc[:, "Grid Node To"] = "ES"
imports_df = imports_df.rename(columns={"BZN|FR > BZN|ES [MW]": "Value"})
imports_df = imports_df[final_columns]

exchanges_df = pd.concat([exports_df, imports_df])
exchanges_df.loc[:, "unit"] = "MWh"

exchanges_df.to_csv("imports_exports.csv", index=False)

In [6]:
# Load forecast
load_forecast_df = pd.read_csv("raw/Total Load - Day Ahead _ Actual_202211010000-202211020000.csv")

load_forecast_df.loc[:, "Grid Node"] = "ES"
load_forecast_df.loc[:, "datetime"] = load_forecast_df.loc[:, "Time (UTC)"].apply(lambda x: arrow.get(x.split(" - ")[0], "DD.MM.YYYY HH:mm").format("YYYY-MM-DD HH:mm"))
load_forecast_df = load_forecast_df.drop(columns=["Time (UTC)"])
load_forecast_df = load_forecast_df.rename(columns={"Day-ahead Total Load Forecast [MW] - BZN|ES": "Demand Forecast (MW)"})

load_forecast_df.to_csv("load_forecast.csv", index=False)